# Federated Multi Layer Perceptron using PyTorch

In [1]:
import warnings
import syft as sy
import torch as th
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\mans_\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


Hook that extends the Pytorch library to enable all computations with pointers of tensors sent to other workers

In [2]:
hook = sy.TorchHook(th)
th.cuda.is_available()

True

### Pre-processing of the data

In [3]:
df = pd.read_csv("creditcard.csv")
df.head()

# Feature scaling 
df['normalizedAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))
df = df.drop(['Amount'],axis=1)
df = df.drop(['Time'],axis=1)

# Split the data into training and test set
X = df.iloc[:, df.columns != 'Class']
y = df.iloc[:, df.columns == 'Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=42)

print(X_train.shape)
print(X_test.shape)

(170883, 29)
(56962, 29)


Turning the data into PyTorch format

In [4]:
# Converting to PyTorch tensors
y_train_tensor = th.tensor(y_train.values).float()
X_train_tensor = th.tensor(X_train.values).float()
y_test_tensor = th.tensor(y_test.values).float()
X_test_tensor = th.tensor(X_test.values).float()
y_val_tensor = th.tensor(y_val.values).float()
X_val_tensor = th.tensor(X_val.values).float()

# Converting to tensor dataset
train = TensorDataset(X_train_tensor, y_train_tensor)
test = TensorDataset(X_test_tensor, y_test_tensor)
val = TensorDataset(X_val_tensor, y_val_tensor)

# Converting to dataloaders 
# (, drop_last=True)
test_loader = DataLoader(test, batch_size=32)
val_loader = DataLoader(val, batch_size = 32)

### Modify the data for FL

Creating virtual workers

In [5]:
client1 = sy.VirtualWorker(hook, id='client_1')
client2 = sy.VirtualWorker(hook, id='client_2')
client3 = sy.VirtualWorker(hook, id='client_3')
client4 = sy.VirtualWorker(hook, id='client_4')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [6]:
client1._objects

{}

Federated Data Loader

In [7]:
# The .federated method splits the dataset in two parts and send them to the virtual workers. 
# This federated dataset is now given to a Federated DataLoader which will iterate over remote batches.
federated_train_loader = sy.FederatedDataLoader(
    train.federate((client1, client2, client3, client4)), batch_size=32, shuffle=True)
    

# Check that our trainloader returns a pointer to a batch, and that transformations are applied
data, labels = next(iter(federated_train_loader))
data

(Wrapper)>[PointerTensor | me:26415390476 -> client_1:37645800325]

In [8]:
print('Client 1: {}, Client 2: {}, Client 3: {}, Client 4: {}'.format(len(client1._objects), 
                                                                      len(client2._objects), 
                                                                      len(client3._objects), 
                                                                      len(client4._objects)))

Client 1: 4, Client 2: 2, Client 3: 2, Client 4: 2


### Creating the model and evaluation

In [9]:
# A class for the MLP model with 3 layers (1 input, 1 hidden and 1 output)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.input = nn.Linear(29,15)
        self.hidden = nn.Linear(15,15)
        self.output = nn.Linear(15, 1)

    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = th.sigmoid(self.output(x))
        return x

### Functions to train the model in a federated learning setting

In [10]:
def client_model_optimizer(client, model):
    """ Copy the global model and send it to a client and also initialize the optimizer for this client. 
    IN: client
    OUT: client_model, client_opt
    """
    # Copy the model and send it to client 'no.'
    client_model = model.copy().send(client)

    # Initialize the optimizers, the learnable parameters of a model are returned by net.parameters
    client_opt = optim.SGD(params=client_model.parameters(), lr=0.02)
    
    return client_model, client_opt

In [11]:
def train_each_client(client_opt, client_model, data, target):
    """  1. Zero out the gradients
         2. Make a forward pass
         3. Calculate the loss with BCE loss 
         4. Backpropagate
         5. Make an optimizer step to update the weights 
         6. Get the loss
    IN: Optimizer, model, data and target for each batch
    OUT: Updated model, loss and optimizer
    """  

    client_opt.zero_grad()
    client_output = client_model.forward(data)
    client_loss = criterion(client_output, target)
    client_loss.backward()
    client_opt.step()
    client_loss = client_loss.get().data
    
    return client_model, client_loss, client_opt

In [12]:
def push_data_to_central_server(client_models, model):
    """ Pushing the data from each client to the central server and then taking an average of these.
    IN: client_models (list of each clients model)
    OUT: model 
    """
    no_clients = len(client_models)
    
    sum_input_weight_data = 0
    sum_input_bias_data = 0
    
    sum_hidden_weight_data = 0
    sum_hidden_bias_data = 0
    
    sum_output_weight_data = 0
    sum_output_bias_data = 0
    
    # Extract each client and its model respectively
    for client in client_models:
        client.get()
        
        sum_input_weight_data += client.input.weight.data
        sum_input_bias_data += client.input.bias.data

        sum_hidden_weight_data += client.hidden.weight.data
        sum_hidden_bias_data += client.hidden.bias.data

        sum_output_weight_data += client.output.weight.data
        sum_output_bias_data += client.output.bias.data
            
    with th.no_grad():
        model.input.weight.set_(sum_input_weight_data / no_clients)
        model.input.bias.set_(sum_input_bias_data / no_clients)

        model.hidden.weight.set_(sum_hidden_weight_data / no_clients)
        model.hidden.bias.set_(sum_hidden_bias_data / no_clients) 

        model.output.weight.set_(sum_output_weight_data / no_clients)
        model.output.bias.set_(sum_output_bias_data / no_clients)
    
    return model

In [13]:
def loss_curve(train_loss, val_loss, epochs):
    """ Plot loss against epochs.
    IN: train_loss (list float), val_loss (list float), epochs (list int)
    OUT: - 
    """
    plt.plot(train_loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    

In [14]:
def loss_prob_model(model, loader, X_tensor):
    """ Computes the loss and probabilities in order to compute the AUPCR score.
    IN: model, loader (either test- or validation), X_tensor (tensor for either test- or validation)
    OUT: output_list (a list of probabilities)
    """
    model.eval()
    loss = 0
    output_list =[]
    
    for data, target in loader:
        output = model(data)
        loss += F.binary_cross_entropy(output, target, reduction='sum').item()
        output_list.append(output)
    loss /= len(X_tensor)
    print('Average loss: {:.8f}'.format(loss))
    return output_list

In [15]:
def prediciton_model(model, y, loader, X_tensor):
    """ Test the model on the validation set every 10:th epoch.
    IN: model, y (dataframe: either test- or validationset), loader (either test- or validation), 
    X_tensor (tensor for either test- or validation)
    OUT: - 
    """
    probabilities = loss_prob_model(model, loader, X_tensor)
    
    y_pred = th.cat(probabilities)
    y_pred_numpy = y_pred.detach().numpy() 
    y_pred_binary = (y_pred_numpy > 0.5)
    
    aucpcr = average_precision_score(y, y_pred_numpy)
    
    print('AUPCR score: ' + str(aucpcr))
   # print('AUPCR score: {:.8f}'.format(str(aucpcr)))
    
    cm = confusion_matrix(y, y_pred_binary)
    print(cm)

In [16]:
def compute_aupcr(y, y_pred_numpy):
    """ A function to plot the AUPCR curve.
    IN: y (dataframe: either test- or valiationset), y_pred_numpy (numpy: either test- or validationset)
    OUT: -
    """

    fpr, tpr, threshold = metrics.precision_recall_curve(y,  y_pred_numpy)
    roc_auc = average_precision_score(y, y_pred_numpy)

    plt.title('Precision recall')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.1, 1])
    plt.ylim([0, 1.1])
    plt.ylabel('True Positive Rate (TPR)')
    plt.xlabel('False Positive Rate (FPR)')
    #plt.savefig('AUP_skewed_val.png')
    plt.show()

In [17]:
def training(all_clients, model, federated_loader, y_val, loader, X_tensor, epoch=60, C=1, local_iterations=2):
    """ Training the global model by using the clients local training. 
    IN: clients (list), model, federated_loader (federated data loader), 
    y_val (pandas dataframe), epoch (int), local_iterations (int)
    OUT: - 
    """
    #nbr_local_iter = 0
    nbr_epoch = 0
    
    # Creating a list to store each clients model and optimizer
    #clients = random.sample(all_clients, C)
    #client_models = [0]*len(clients) 
    #client_opt = [0]*len(clients) 
    #client_loss = [0]*len(clients) 
    
    # Number of local epochs for the central model on each client
    for round_iter in range(epoch):
        nbr_epoch += 1
        nbr_local_iter = 0
        
        clients = random.sample(all_clients, C)
        client_models = [0]*len(clients) 
        client_opt = [0]*len(clients) 
        client_loss = [0]*len(clients)
        
        print('All clients', all_clients)
        print('Random clients', clients)
        
        # Copying the model and getting an optimizer for each client and store them in the lists
        for idx, client in enumerate(clients):
            client_mod, client_optimizer = client_model_optimizer(client, model)
            client_models[idx] = client_mod
            client_opt[idx] = client_optimizer

        # Train the models and average them
        for i in range(local_iterations):
            nbr_local_iter += 1 
            
            # Iteration over each batch 
            for (data, target) in federated_loader:
                
                for idx, client in enumerate(clients):
                    # Run through each client to see which one fits 
                    if data.location == client:
                        client_model, client_loss, client_optimizer = train_each_client(client_opt[idx], client_models[idx], data, target)
                        #print('client_loss type: ', type(client_loss))
                        #print('client_loss: ', client_loss)
                        client_models[idx] = client_model
                        #client_loss[idx] = client_loss.item()
                        client_opt[idx] = client_optimizer
            
            # Predictions for each local iteration
            # print('Local iteration: ', nbr_local_iter)
            # prediciton_model(model, y_val, loader, X_tensor)

                
            #print("Number of local iterations: ", nbr_local_iter)

        print("Total number of epochs: ", nbr_epoch)
        
        # Evaluate the model after every 5:th epoch
        if nbr_epoch % 10 == 0:
            print('Epoch: ', nbr_epoch)
            prediciton_model(model, y_val, loader, X_tensor)
            
        # Pushing the data from each client to the central server and averaging these (we are not using secure worker yet)
        model = push_data_to_central_server(client_models, model)
            
        secure_worker.clear_objects()

        #print("Client 1:" + str(client_loss[0]) + " Client 2:" + str(client_loss[1]))

### Create and train the model on the validationset

In [18]:
import random

model = Net()
criterion = nn.BCELoss()
clients = [client1, client2, client3, client4]
training(clients, model, federated_train_loader, y_val, val_loader, X_val_tensor, epoch=60, C=2, local_iterations=2)

All clients [<VirtualWorker id:client_1 #objects:4>, <VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:client_3 #objects:2>, <VirtualWorker id:client_4 #objects:2>]
Random clients [<VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:client_3 #objects:2>]
Total number of epochs:  1
All clients [<VirtualWorker id:client_1 #objects:4>, <VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:client_3 #objects:2>, <VirtualWorker id:client_4 #objects:4>]
Random clients [<VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:client_4 #objects:4>]
Total number of epochs:  2
All clients [<VirtualWorker id:client_1 #objects:4>, <VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:client_3 #objects:2>, <VirtualWorker id:client_4 #objects:4>]
Random clients [<VirtualWorker id:client_4 #objects:4>, <VirtualWorker id:client_1 #objects:4>]
Total number of epochs:  3
All clients [<VirtualWorker id:client_1 #objects:4>, <VirtualWorker id:client_2 #objects:2>, <VirtualWorker id:c